Задание:
1. Снять видео на камеру (любую, можно телефона). Выполнить кручение видео с скоростью 10 градусов в секунду относительно центра (в любую сторону). 
2. Видео после выполнения п.1 и вернуть его в исходное состояние. 
3. Сконвертируйте видео в mp4 при помощи ffmpeg

In [1]:
import cv2
import numpy as np
from tqdm import tqdm
import pandas as pd

In [35]:
def get_restored_matrix(to_int8_matrix):
    scale = 0.01166
    offset = -348.91
    to_int8_matrix_rec = []
    for i in range(0, len(to_int8_matrix), 2):
        to_int8_matrix_rec.append((to_int8_matrix[i] << 8) | to_int8_matrix[i+1])
    int16_x = np.matrix(to_int8_matrix_rec)
    int16_x = int16_x.reshape(2,3)
    restored_matrix = int16_x.astype(np.float32) * scale + offset
    return restored_matrix
    

In [31]:
def get_INT_matrix(matrix):
    # Исходная матрица
    '''
    matrix = np.array([
        [9.84807753e-01, 1.73648178e-01, -3.48919959e+02],
        [-1.73648178e-01, 9.84807753e-01, 4.15826615e+02]
    ])
'''
    # Шаг 1: Нормализация для приведения значений к диапазону int8
    matrix_min = matrix.min()
    matrix_max = matrix.max()

    # Вычисление коэффициента масштабирования
    scale = (matrix_max - matrix_min) / (32767 - (-32768))  # Диапазон int8: от -128 до 127
    offset = matrix_min  # Сохраняем смещение (минимальное значение)

    # Преобразование в int16
    normalized_matrix = ((matrix - offset) / scale).astype(np.int16)
    
    to_int8_matrix = []
    for i in normalized_matrix:
        for item in i:
            high_byte = (item >> 8) & 0xFF  # Старший байт (сдвиг на 8 бит вправо и маска 0xFF)
            low_byte = item & 0xFF          # Младший байт (маска 0xFF)
            to_int8_matrix.append(high_byte)
            to_int8_matrix.append(low_byte)
    int8_matrix = np.matrix(to_int8_matrix).astype(np.int8)
    return int8_matrix
    ''' 
    to_int8_matrix_rec = []
    for i in range(0, len(to_int8_matrix), 2):
        to_int8_matrix_rec.append((to_int8_matrix[i] << 8) | to_int8_matrix[i+1])
    int16_x = np.matrix(to_int8_matrix_rec)
    int16_x = int16_x.reshape(2,3)
    
    
    #return normalized_matrix

    # Шаг 2: Сохранение scale и offset (для восстановления)

    # Шаг 3: Восстановление матрицы
    restored_matrix = int16_x.astype(np.float32) * scale + offset

# Вывод результатов
    print("Исходная матрица:")
    print(matrix)
    print("\nПреобразованная матрица (int8):")
    print(normalized_matrix)
    print("\nВосстановленная матрица:")
    print(restored_matrix)
    print("toint матрица:")
    print(to_int8_matrix)
    print("Восстановлена из toint матрица:")
    print(to_int8_matrix_rec)
'''


In [53]:
def rotate_image_border(image, angle):
    center = (image.shape[1] // 2, image.shape[0] // 2)  # Центр вращения
    height, width = image.shape[:2]
    diag = int(np.sqrt(width**2 + height**2))  # Диагональ (наибольшая сторона после вращения)

    # Добавление чёрных рамок, чтобы вместить всё изображение после поворота
    border_x = (diag - width) // 2
    border_y = (diag - height) // 2
    image_with_border = cv2.copyMakeBorder(image, border_y, border_y, border_x, border_x, cv2.BORDER_CONSTANT, value=(0, 0, 0))

    # Обновление центра вращения из-за рамок
    new_center = (image_with_border.shape[1] // 2, image_with_border.shape[0] // 2)
    
    rot_mat = cv2.getRotationMatrix2D(new_center, angle, 1.0)
    

# Применение вращения
    rotated_image = cv2.warpAffine(image_with_border, rot_mat, (diag, diag), flags=cv2.INTER_LINEAR)
    frame = cv2.resize(rotated_image, (512, 640))
    return frame

In [33]:
def rotate_image_border_with_data(image, angle):
    center = (image.shape[1] // 2, image.shape[0] // 2)  # Центр вращения
    height, width = image.shape[:2]
    diag = int(np.sqrt(width**2 + height**2))  # Диагональ (наибольшая сторона после вращения)

    # Добавление чёрных рамок, чтобы вместить всё изображение после поворота
    border_x = (diag - width) // 2
    border_y = (diag - height) // 2
    image_with_border = cv2.copyMakeBorder(image, border_y, border_y, border_x, border_x, cv2.BORDER_CONSTANT, value=(0, 0, 0))

    # Обновление центра вращения из-за рамок
    new_center = (image_with_border.shape[1] // 2, image_with_border.shape[0] // 2)
    
    rot_mat = cv2.getRotationMatrix2D(new_center, angle, 1.0)
    

# Применение вращения
    rotated_image = cv2.warpAffine(image_with_border, rot_mat, (diag, diag), flags=cv2.INTER_LINEAR)
    frame = cv2.resize(rotated_image, (512, 641))
    matrix_int = get_INT_matrix(rot_mat)
    #print(matrix_int)
    
    frame [640,0] = matrix_int[0,0:3]
    frame [640,1] = matrix_int[0,3:6]
    frame [640,2] = matrix_int[0,6:9]
    frame [640,3] = matrix_int[0,9:12]
    return frame
   # print (frame)
    '''
    print (rot_mat)
    print ('______________')
    
    print(frame [640,0])
    print(frame [640,1])
    print(frame [640,2])
    print(frame [640,3])
    print ('______________')
   # print (frame.shape)
   
    return frame
   '''

In [36]:
def stop_rotate(frame):
    
    rot_mat = get_restored_matrix (frame)
    rot_mat_3x3 = np.vstack([rot_mat, [0, 0, 1]])
# Находим обратную матрицу
    inv_rot_mat_3x3 = np.linalg.inv(rot_mat_3x3)
# Возвращаем обратно в 2x3
    inv_rot_mat = inv_rot_mat_3x3[:2, :]
  
    result = cv2.warpAffine(frame, inv_rot_mat, frame.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

In [6]:
def rotate_image(image, angle):
  image_center = tuple(np.array(image.shape[1::-1]) / 2)
  rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
  result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
  return result

In [7]:
def rotate_image_reverse(image, angle):
  image_center = tuple(np.array(image.shape[1::-1]) / 2)

  rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
  
  rot_mat_3x3 = np.vstack([rot_mat, [0, 0, 1]])
# Находим обратную матрицу
  inv_rot_mat_3x3 = np.linalg.inv(rot_mat_3x3)
# Возвращаем обратно в 2x3
  inv_rot_mat = inv_rot_mat_3x3[:2, :]
  
  result = cv2.warpAffine(image, inv_rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
  return result

In [61]:
def read_and_save_rotate (read_name,out_name,out_format,angle, DEBUG = 1, rotate_function = rotate_image): 
  cap = cv2.VideoCapture(read_name)


  output_filename = out_name
  frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
  fps = int(cap.get(cv2.CAP_PROP_FPS))
  frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))



  fourcc = cv2.VideoWriter_fourcc(*out_format)
  out = cv2.VideoWriter(output_filename, fourcc, fps, (512, 640))


  rotate_angle = 0
  frame_counter = 0


  # Check if camera opened successfully
  if (cap.isOpened()== False): 
    print("Error opening video stream or file")
    

    
  # Read until video is completed
  while(cap.isOpened()):
    # Capture frame-by-frame
      
    ret, frame = cap.read()
      

      
    if frame_counter%fps == 0:
        rotate_angle += angle
      
    if rotate_angle == 360:
        rotate_angle == 0
      #print(frame.shape)
      
      
    if ret == True:
        
      #frame = cv2.resize(frame, (output_width, output_height))
      frame_counter+=1
      
      frame = rotate_function(frame, rotate_angle)
     # frame = cv2.resize(frame, (512, 640))
      out.write(frame)
      if DEBUG:
        # Display the resulting frame
        cv2.imshow('window',frame)
      
      # Press Q on keyboard to  exit
      if cv2.waitKey(25) & 0xFF == ord('q') or frame_counter == 240:
        break
        
    
      # Break the loop
    else: 
      break
  
  # When everything done, release the video capture object
  cap.release()
  out.release()
  # Closes all the frames
  cv2.destroyAllWindows()

In [62]:
read_and_save_rotate(read_name='IMG_6210.hevc',out_name='output.mov',out_format='Moov',angle = 10, DEBUG = 1, rotate_function=rotate_image_border)


In [ ]:
def get_INT_matrix(matrix):
    # Исходная матрица
    matrix = np.array([
        [9.84807753e-01, 1.73648178e-01, -3.48919959e+02],
        [-1.73648178e-01, 9.84807753e-01, 4.15826615e+02]
    ])

    # Шаг 1: Нормализация для приведения значений к диапазону int8
    matrix_min = matrix.min()
    matrix_max = matrix.max()

    # Вычисление коэффициента масштабирования
    scale = (matrix_max - matrix_min) / (32767 - (-32768))  # Диапазон int8: от -128 до 127
    offset = matrix_min  # Сохраняем смещение (минимальное значение)

    # Преобразование в int16
    normalized_matrix = ((matrix - offset) / scale).astype(np.int16)
    return normalized_matrix

    # Шаг 2: Сохранение scale и offset (для восстановления)
'''
    # Шаг 3: Восстановление матрицы
    restored_matrix = normalized_matrix.astype(np.float32) * scale + offset

# Вывод результатов
print("Исходная матрица:")
print(matrix)
print("\nПреобразованная матрица (int8):")
print(normalized_matrix)
print("\nВосстановленная матрица:")
print(restored_matrix)
'''

In [37]:
read_and_save_rotate (read_name='output.mov',out_name='original_convert.mp4',out_format='mp4v',angle = 10, DEBUG = 1, rotate_function=rotate_image_reverse)

Error opening video stream or file


In [11]:
cap = cv2.VideoCapture('output.mp4')


output_filename = 'original.mp4'
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))


fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_filename, fourcc, fps, (frame_width, frame_height))
rotate_angle = 0
frame_counter = 0
# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")
# Read until video is completed
while(cap.isOpened()):
  # Capture frame-by-frame
  
  ret, frame = cap.read()
  
  
  '''
  height, width = frame.shape[:2]
  scale = 0.2 # Масштаб
  output_width = int(width * scale)
  output_height = int(height * scale)
  '''
  
  
  #original_fps = cap.get(cv2.CAP_PROP_FPS)
 # print(f"FPS видео: {original_fps}")
  if frame_counter%fps == 0:
      rotate_angle += 10
  
  if rotate_angle == 360:
      rotate_angle == 0
  #print(frame.shape)
  
  
  if ret == True:
    
    #frame = cv2.resize(frame, (output_width, output_height))
    frame_counter+=1

    frame = rotate_image_reverse(frame, rotate_angle)
    
    out.write(frame)

    # Display the resulting frame
    cv2.imshow('Frame',frame)
 
    # Press Q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break
    
 
  # Break the loop
  else: 
    break
 
# When everything done, release the video capture object
cap.release()
out.release()
# Closes all the frames
cv2.destroyAllWindows()

KeyboardInterrupt: 